In [ ]:
import os

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import cv2
from tqdm.auto import tqdm

import torch
from torch import nn
import torchvision.models as models 
from torch.utils.data import Dataset, DataLoader
import torchvision

In [ ]:
FPS = 50
EPOCHS = 30
IMAGE_SIZE = 128
SEED = 22
LEARNING_RATE = 1e-4

In [ ]:
class CustomDataset():
  def __init__(self, path, video_path, labels, transform=None):
    self.path = path
    self.video_path = video_path
    self.labels = labels

  def __len__(self):
    return len(self.video_path)

  def __getitem__(self, idx):
    video_path = self.path + self.video_path[idx][2:]
    frames = self.video_frames(video_path)
    if self.labels is not None:
      label = self.labels[idx]
      return frames, label
    else:
      return frames

  def video_frames(self, video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    for i in range(FPS):
      _, img = cap.read()
      img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
      img = img / 225
      frames.append(img)
    return torch.FloatTensor(np.array(frames)).permute(3, 0, 1, 2)

In [ ]:
test_csv = pd.read_csv('/content/drive/MyDrive/dacon/car_crash/test.csv')
test_csv.tail()

In [ ]:
class TrainedModel(nn.Module):
  def __init__(self, num_classes):
    super(TrainedModel, self).__init__()
    self.backbone = models.video.r3d_18(weights='KINETICS400_V1')
    self.fc = nn.Linear(400, num_classes)

  def forward(self, x):
    x = self.backbone(x)
    x = self.fc(x)
    return x

In [ ]:
crash = TrainedModel(2)
crash.load_state_dict(torch.load('/content/drive/MyDrive/dacon/car_crash/best_crash_model.pt'))

ego = TrainedModel(2)
ego.load_state_dict(torch.load('/content/drive/MyDrive/dacon/car_crash/best_ego_model.pt'))

# crash_ego = TrainedModel(3)
# crash_ego.load_state_dict(torch.load('/content/drive/MyDrive/dacon/car_crash/crash_ego_weather_model.pt'))

weather = TrainedModel(3)
weather.load_state_dict(torch.load('/content/drive/MyDrive/dacon/car_crash/best_weather_model3.pt'))

time = TrainedModel(2)
time.load_state_dict(torch.load('/content/drive/MyDrive/dacon/car_crash/best_time_model.pt'))

In [ ]:
test_x = test_csv.iloc[:, 1].values

In [ ]:
test_dataset = CustomDataset('/content/drive/MyDrive/dacon/car_crash/', test_x, None)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [ ]:
crash_preds = inference(crash, test_dataloader, device)
ego_preds = inference(ego, test_dataloader, device)
# crash_ego_preds = inference(crash_ego, test_dtaloader, device)
weather_preds = inference(weather, test_dataloader, device)
time_preds = inference(time, test_dataloader, device)

In [ ]:
label_dict = {
    '1100':1,
    '1101':2,
    '1110':3,
    '1111':4,
    '1120':5,
    '1121':6,
    '1000':7,
    '1001':8,
    '1010':9,
    '1011':10,
    '1020':11,
    '1021':12
}

In [ ]:
result = []

for c,e,w,t in zip(crash_preds, ego_preds, weather_preds, time_preds):
  if c == 0:
    result.append(0)
  else:
    result.append(label_dict[str(c)+str(e)+str(w)+str(t)])

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/dacon/car_crash/sample_submission.csv')
submit['label'] = result
submit.to_csv('/content/drive/MyDrive/dacon/car_crash/r_3d_18_each_model_predict14.csv', index=False)